In [1]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
from algorithms.autoencoder import autoencoder as ae
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder as onehot

#path = "/media/jahan/solo/"
#os.chdir(path)

#labels = pd.read_csv("CheXpert-v1.0-small/train.csv")
#print(len(labels))
#labels = labels[labels["Frontal/Lateral"] == "Frontal"]
#print(len(labels))


all_image_paths = "labels.Path.values"

labels = [0,1,1]
#[1 if x == 1 else 0 for x in labels["No Finding"]]
#encode = onehot()

#labels = encode.fit_transform(np.array(labels).reshape((-1,1))).toarray()
#print(labels[:5])

#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#sess = tf.Session(config=config)


save_path = "/home/jahan/Documents/CXR_CNN/output/"


encoder = ae(image_paths=all_image_paths, labels=labels, save_path=save_path)
#encoder.build_dataset(batch_size=32)
encoder.build()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            [(None, 256, 256, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 61) 122         input_1[0][0]                    
______________________________________________________________________________________________